In [3]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [14]:
import logging
import delta_sharing
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [5]:
customer = "Kfzteile24"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [6]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.{account_id}.conversion_probabilities_view_30_outlook"#_6819bcae879ec1741090bc71"
df = delta_sharing.load_as_pandas(table_path)#, limit=1000)

In [7]:
df

In [8]:
df["date"].min(), df["date"].max()

In [9]:
df.columns

In [17]:
df = df.sort_values(by="date")

In [18]:
df["date"].value_counts()

In [19]:
df["probability"].describe()

In [26]:
sns.lineplot(x="date",y="probability",hue="goal",data=df)
plt.ylim(0,1)
plt.xticks(rotation=90)


In [27]:
stats_day = df.groupby(by=["date","goal"])["probability"].describe().reset_index()
stats_day

In [31]:
sns.lineplot(x="date",y="mean",data=stats_day, hue="goal")#, label="mean")
sns.lineplot(x="date",y="min",data=stats_day, hue="goal")#, label="min")
sns.lineplot(x="date",y="50%",data=stats_day, hue="goal")#, label="50%")
plt.ylim(0,1)
plt.xticks(rotation=90)

In [25]:
df["goal"].value_counts(dropna=False)

# For different goals

In [32]:
profile_path = db_client.return_databricks_client()
df_goals = pd.DataFrame()
for goal in ["67ceb86f2eb78f8bcc0b643d","681a0bbe879ec174109e6d13"]:
    table_path = f"{profile_path}#delta_share_events.{account_id}.conversion_probabilities_view_90_outlook_{goal}"#_6819bcae879ec1741090bc71"
    temp = delta_sharing.load_as_pandas(table_path)#, limit=1000)
    print(temp["goal"].value_counts(dropna=False))
    df_goals = pd.concat([df_goals,temp])
df_goals["goal"].value_counts(dropna=False)
    

In [33]:
stats_day_goal = df_goals.groupby(by=["date","goal"])["probability"].describe().reset_index()
stats_day_goal

In [34]:
sns.lineplot(x="date",y="mean",data=stats_day_goal, hue="goal")#, label="mean")
sns.lineplot(x="date",y="min",data=stats_day_goal, hue="goal")#, label="min")
sns.lineplot(x="date",y="50%",data=stats_day_goal, hue="goal")#, label="50%")
plt.ylim(0,1)
plt.xticks(rotation=90)